In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1442.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  355.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4105 personas en los últimos 7 días 

Un positivo PCR cada 1710 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] != 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1442.0,234.0,408.0,510743.0,45.815606,79.883621,41.0
Huelva-Costa,746.0,94.0,195.0,288115.0,32.625861,67.681308,21.0
Condado-Campiña,465.0,104.0,156.0,155057.0,67.072109,100.608163,19.0
Palma del Condado (La),64.0,28.0,32.0,10761.0,260.198866,297.370133,10.0
Lepe,99.0,24.0,32.0,27431.0,87.492253,116.656338,7.0
Gibraleón,21.0,5.0,9.0,12607.0,39.660506,71.388911,4.0
Huelva (capital),355.0,35.0,84.0,143663.0,24.362571,58.470170,4.0
Villanueva de los Castillejos,6.0,4.0,4.0,2820.0,141.843972,141.843972,3.0
Ayamonte,50.0,5.0,20.0,20946.0,23.870906,95.483625,3.0


In [14]:
if tabla[tabla['Diferencia PCR'] != 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Campillo (El),26.0,24.0,24.0,2023.0,1186.356896,1186.356896,0.0
Manzanilla,13.0,9.0,9.0,2135.0,421.545667,421.545667,0.0
Almendro (El),4.0,3.0,3.0,826.0,363.196126,363.196126,0.0
Bollullos Par del Condado,84.0,33.0,48.0,14272.0,231.221973,336.322870,1.0
Palma del Condado (La),64.0,28.0,32.0,10761.0,260.198866,297.370133,10.0
Granado (El),2.0,1.0,1.0,516.0,193.798450,193.798450,0.0
Jabugo,5.0,1.0,4.0,2250.0,44.444444,177.777778,0.0
Aroche,9.0,1.0,5.0,3073.0,32.541490,162.707452,0.0
Villanueva de los Castillejos,6.0,4.0,4.0,2820.0,141.843972,141.843972,3.0
